# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations
locations = cities_df[["Lat","Lng"]]

# Add humidity layer to map
humidity = cities_df["Humidity"]

In [4]:
# Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
# Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
fig.add_layer(heat_layer)

# Display fig
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_filtered = cities_df.loc[(cities_df["Wind Speed"] <= 20) & (cities_df["Cloudiness"] <= 50) & (cities_df["Max Temp"] >= 60) & (cities_df["Max Temp"] <= 80)].dropna()
cities_filtered

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
22,22,ambon,11,FR,1558378758,93,47.55,-2.56,64.99,5.82
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
...,...,...,...,...,...,...,...,...,...,...
522,522,raudeberg,20,NO,1558378894,76,61.99,5.14,69.80,4.70
528,528,lyuban,39,BY,1558378625,68,52.80,28.00,67.92,12.15
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = cities_filtered.loc[:,["City","Country","Lat","Lng"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Disply df
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,flinders,AU,-34.58,150.85,
9,pushkinskiye gory,RU,57.02,28.91,
22,ambon,FR,47.55,-2.56,
25,kantang,TH,7.42,99.52,
37,varhaug,NO,58.61,5.65,
...,...,...,...,...,...
522,raudeberg,NO,61.99,5.14,
528,lyuban,BY,52.80,28.00,
533,hakkari,FI,61.34,23.75,
538,la seyne-sur-mer,FR,43.10,5.88,


In [7]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No hotel found")

Closest hotel in flinders is Lakeview Hotel Motel.
Closest hotel in pushkinskiye gory is Baza Otdykha "Zaozer'ye".
Closest hotel in ambon is Auberge de Pen-Mur.
Closest hotel in kantang is Kantang Loftel.
Closest hotel in varhaug is Huset ved havet.
Closest hotel in puerto ayora is Galapagos Habitat by Eco Luxury Group.
Closest hotel in pisco is PiscoMar Peru.
Closest hotel in beja is Pousada Convento Beja.
Closest hotel in fortuna is Balneario de Leana.
Closest hotel in ponta do sol is Escarpas Resort.
No hotel found
Closest hotel in crotone is Hotel San Giorgio.
Closest hotel in emerald is Emerald Maraboon Motor Inn.
No hotel found
Closest hotel in ambilobe is Hotel National.
Closest hotel in cidreira is Hotel Castelo.
Closest hotel in antioch is Best Western Plus Delta Inn & Suites.
Closest hotel in hilo is Reeds Bay Resort Hotel.
Closest hotel in whitehorse is Best Western Gold Rush Inn.
Closest hotel in koumac is Hotel Karem Bay.
Closest hotel in sao filipe is Casas Do Sol.
Closes

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display fig
fig

Figure(layout=FigureLayout(height='420px'))